In [ ]:
import scipy.stats as stats
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt

In [ ]:
table = pd.DataFrame()

for point in np.arange(0, 3.10, step = .01):
    
    single_dec = math.trunc(point * 10) / 10
    double = math.trunc(point * 100 + .0001) / 100  # +.0001 for float margin of error
    assert double >= single_dec
    hundreds = double - single_dec
    cdf_val = stats.norm.cdf(double)
    
    is_even = (10*single_dec % 2) == 0
    
    helper = ''
    if is_even:
        helper = r'\cellcolor{gray!20}'
    
    table.loc[str(single_dec), str('{:+.2f}'.format(hundreds))] = helper + '{:.4f}'.format(cdf_val)
    
assert table.isnull().sum().sum() == 0

In [ ]:
#print(table.to_latex(escape = False))
s = table.style.to_latex(hrules = True)

In [ ]:
wd = os.getcwd().rstrip("/python")

%cd ~

In [ ]:
# Add illustration

x = np.linspace(-3,3, 100_000)
fig, ax = plt.subplots()
y = stats.norm.pdf(x)
ax.plot(x, y)
ax.set_aspect(6)

for spine in 'left', 'right', 'top':
    ax.spines[spine].set_visible(False)
ax.set_yticks([])
ylim = ax.get_ylim()
xlim = ax.get_xlim()

z_score = 1
x_vals = x[x<z_score]
y_vals = y[0:len(x_vals)]
ax.fill_between(x_vals, y_vals, alpha = 0.15)

x = np.linspace(-4,4, 200_000)
y = stats.norm.pdf(x)
ax.plot(x, y, color = 'C0')

ax.set_ylim(0, ylim[1])
ax.set_xlim(xlim)

# annotate area

ax.text(1, stats.norm.pdf(1)/1.2,
        s = '{:.4f}  '.format(stats.norm.cdf(1)),
        ha = 'right',
        va = 'center')

ax.set_xticks([1])
ax.set_xticklabels(['$z=1$'])

ax.plot([1,1], [0, stats.norm.pdf(1)], color = 'black', ls = 'dashed')
#ax.plot([1], [stats.norm.pdf(1)], color = 'black', marker = '.')

plt.savefig('ztable_plot.pdf')
plt.show()

In [ ]:
with open("ztable-input.tex", 'w') as f:
    f.write(s)
    
with open("ztable.tex", 'w') as f:
    doc = r"""\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage[margin=1in]{geometry}
\usepackage{booktabs}
\usepackage{colortbl}
\usepackage{graphicx}
\usepackage{xcolor}
\begin{document}
\begin{center}
\section*{$z$-table}
%\input{ztable-input.tex}
""" + s + r"""
\bigskip
\includegraphics[width=.6\textwidth]{ztable_plot.pdf}
\end{center}
\end{document}
"""
    f.write(doc)

os.system("pdflatex ztable.tex")
os.system("open ztable.tex")
os.system(f"mv ztable.tex {wd}/ztable.tex")
os.system(f"mv ztable.pdf {wd}/ztable.pdf")
os.system(f"mv ztable-input.tex {wd}/tex/ztable-input.tex")

files = !ls
for f in files:
    if f.startswith("ztable."):
        os.system(f"rm {f}")
        #print(f)